In [ ]:
import pandas as pd
import warnings; 
warnings.simplefilter('ignore')
#Yahoo Finance API
import yahoo_fin.stock_info as yfi
tickers = yfi.tickers_sp500()
import yfinance as yf
# Prophet
from prophet import Prophet
from prophet.plot import plot_plotly
# Interactive plots
import plotly.io as pio
pio.renderers.default = "notebook_connected"



In [ ]:
# Reading the data.
def  get_data(ticker, start_date):
    stock = yfi.get_data(ticker, start_date = 2022/0o4/0o1, end_date = None, index_as_date = True, interval = "1d")
    stock["date"] = pd.to_datetime(stock.index)
    stock.reset_index(inplace=True)
    stock_data = stock[["close",  "date"]]
    stock_data.columns = ['y', 'ds']
    return stock_data
#print(stock.head())
#print(stock.dtypes)
#print(stock.open)
#print(stock.index)
#stock_prices = pd.read_csv("stock_prices.csv")
#stock_prices.head()

# Forecasting with Prophet

In [ ]:
#Getting the last 20 days to test our forecast.
last_prices = stock_data[len(stock_data)-20:]
last_prices

In [ ]:
# Get al the rows ecxept the last 20 to train the model.
data = stock_data[:-20]
data.shape

In [ ]:
# Create the model
m = Prophet(daily_seasonality=True)
# Train the model
model = m.fit(data)
future =  model.make_future_dataframe(periods=365)
forecast = model.predict(future)

In [ ]:
#ploting
plot_plotly(model, forecast)

In [ ]:
# Show model prediction for 05/05/2022
forecast[forecast.ds == "2022-05-05"]["yhat"]

In [ ]:
# Show the actual value
last_prices[last_prices.ds == "2022-05-05"]["y"]

In [ ]:

def prophet(stock_data):
    m = Prophet(interval_width=0.95, daily_seasonality=True)
    model = m.fit(stock_data)
    future = m.make_future_dataframe(periods=100,freq='D')
    forecast = m.predict(future)
    return forecast

In [ ]:
forecast = prophet(stock_data)
forecast.tail()

In [ ]:
plot1 = m.plot(forecast)

In [ ]:

plt2 = m.plot_components(forecast)

params = ProphetParams(seasonality_mode='multiplicative') # additive mode gives worse results